In [1]:
import pandas as pd 
import numpy as np
from tqdm.auto import tqdm
import torch 
import torch.nn as nn
import torch.optim as optim 
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer , DistilBertModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from transformers import BertModel, BertTokenizer


In [2]:
with open("Constraint_English_Train - Sheet1.csv") as f:
    data_train = f.readlines()

In [3]:
data_train[:5]

['id,tweet,label\n',
 '1,The CDC currently reports 99031 deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stands at roughly 100000 people today.,real\n',
 '2,States reported 1121 deaths a small rise from last Tuesday. Southern states reported 640 of those deaths. https://t.co/YASGRTT4ux,real\n',
 '3,Politically Correct Woman (Almost) Uses Pandemic as Excuse Not to Reuse Plastic Bag https://t.co/thF8GuNFPe #coronavirus #nashville,fake\n',
 '4,#IndiaFightsCorona: We have 1524 #COVID testing laboratories in India and as on 25th August 2020 36827520 tests have been done : @ProfBhargava DG @ICMRDELHI #StaySafe #IndiaWillWin https://t.co/Yh3ZxknnhZ,real\n']

In [4]:
data_train=pd.read_csv("Constraint_English_Train - Sheet1.csv")
data_test=pd.read_csv("Constraint_English_Test - Sheet1.csv")
data_val=pd.read_csv("Constraint_English_Val - Sheet1.csv")

In [ ]:
data_train.isna().sum()

id       0
tweet    0
label    0
dtype: int64

In [26]:
from sklearn.preprocessing import LabelEncoder
l_enc = LabelEncoder()
data_train["label_1"]=l_enc.fit_transform(data_train["label"])
# data_test["label_1"]=l_enc.fit_transform(data_test["label"])
data_val["label_1"]=l_enc.fit_transform(data_val["label"])

In [ ]:
# 1 pour real 
# 0 pour fake

In [27]:
data_train.head()

,id,tweet,label,label_1
0,1,The CDC currently reports 99031 deaths. In gen...,real,1
1,2,States reported 1121 deaths a small rise from ...,real,1
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake,0
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real,1
4,5,Populous states can generate large case counts...,real,1


In [28]:
data_val.head()

,id,tweet,label,label_1
0,1,Chinese converting to Islam after realising th...,fake,0
1,2,11 out of 13 people (from the Diamond Princess...,fake,0
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake,0
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake,0
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real,1


In [29]:
data_test.head()

,id,tweet
0,1,Our daily update is published. States reported...
1,2,Alfalfa is the only cure for COVID-19.
2,3,President Trump Asked What He Would Do If He W...
3,4,States reported 630 deaths. We are still seein...
4,5,This is the sixth time a global health emergen...


In [31]:
data_train["label_1"].value_counts()

label_1
1    3360
0    3060
Name: count, dtype: int64

In [ ]:
## there s no imbalence imn data

In [35]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\pc\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pc\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [36]:
simple = data_train["tweet"][:10]

In [37]:
simple

0    The CDC currently reports 99031 deaths. In gen...
1    States reported 1121 deaths a small rise from ...
2    Politically Correct Woman (Almost) Uses Pandem...
3    #IndiaFightsCorona: We have 1524 #COVID testin...
4    Populous states can generate large case counts...
5    Covid Act Now found "on average each person in...
6    If you tested positive for #COVID19 and have n...
7    Obama Calls Trump’s Coronavirus Response A Cha...
8    ???Clearly, the Obama administration did not l...
9    Retraction—Hydroxychloroquine or chloroquine w...
Name: tweet, dtype: object

In [43]:
simple.dtype

dtype('O')

In [44]:
simple = str(simple)

In [52]:
type(simple)

str

In [ ]:
encoding = tokenizer.encode_plus(
    simple,
    addspecial_tokens=True,
    max_length=256,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt' 
)


In [55]:
encoding["input_ids"]

tensor([[  101,  1014,  1996, 26629,  2747,  4311,  5585,  2692, 21486,  6677,
          1012,  1999,  8991,  1012,  1012,  1012,  1015,  2163,  2988, 11176,
          2487,  6677,  1037,  2235,  4125,  2013,  1012,  1012,  1012,  1016,
         10317,  6149,  2450,  1006,  2471,  1007,  3594,  6090,  3207,  2213,
          1012,  1012,  1012,  1017,  1001,  2634, 20450,  9363, 26788,  1024,
          2057,  2031, 15017,  2549,  1001,  2522, 17258,  3231,  2378,  1012,
          1012,  1012,  1018, 20151,  2163,  2064,  9699,  2312,  2553,  9294,
          1012,  1012,  1012,  1019,  2522, 17258,  2552,  2085,  2179,  1000,
          2006,  2779,  2169,  2711,  1999,  1012,  1012,  1012,  1020,  2065,
          2017,  7718,  3893,  2005,  1001,  2522, 17258, 16147,  1998,  2031,
          1050,  1012,  1012,  1012,  1021,  8112,  4455,  8398,  1521,  1055,
         21887, 23350,  3433,  1037, 15775,  1012,  1012,  1012,  1022,  1029,
          1029,  1029,  4415,  1010,  1996,  8112,  

In [59]:
encoding['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [61]:
data['label_1'].unique()

array([1, 0])

In [ ]:
import torch.nn as nn 
from torch.utils.data  import Dataset
class Subject_Dataset(Dataset):
    def __init__(self , df):
        self.df=df
        self.maxlen=256
        self.tokinizer=BertTokenizer.from_pretrained("bert-base-uncased")
    def __len__(self):
        return len(self.df)
    def __getitem__(self , index):
        sample= self.df['tweet'].iloc[index]
        sample=' '.join(sample)
        encodings = self.tokenizer.encode_plus(
        sample,
        add_special_tokens=True,
        max_length=256,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt' 
        )
        return {
            'input_ids': encodings['input_ids'].flatten(),  
            'attention_mask': encodings['attention_mask'].flatten(),
            'labels': torch.tensor(self.df['label_1'].iloc[index] , dtype=torch.long)
        } 

In [ ]:
data_train_tokens=CovidDataset(data_train)
data_train_tokens=CovidDataset(data_train)

In [ ]:
## for end to end 
class Fake_News_Derection_Classifier(nn.Module):
    def __init__(self,):
        super(Fake_News_Derection_Classifier, self).__init__()
        self.bert=BertModel.from_pretrained("bert-base-uncased")
        self.drop0 = nn.Dropout(0.25)
        self.linear1=nn.Linear(3071, 512)
        self.relu = nn.ReLU()
        self.drop1=nn.Dropout(0.25)
        self.linear2=nn.Linear(512, 2) # 2 represente the number of 
